In [0]:
!pip install icrawler

In [0]:
from icrawler.builtin import GoogleImageCrawler

# 猫の画像を100枚取得
crawler = GoogleImageCrawler(storage={"root_dir": "cats"})
crawler.crawl(keyword="猫", max_num=100)

2019-07-07 06:34:47,503 - INFO - icrawler.crawler - start crawling...
2019-07-07 06:34:47,504 - INFO - icrawler.crawler - starting 1 feeder threads...
2019-07-07 06:34:47,511 - INFO - feeder - thread feeder-001 exit
2019-07-07 06:34:47,512 - INFO - icrawler.crawler - starting 1 parser threads...
2019-07-07 06:34:47,519 - INFO - icrawler.crawler - starting 1 downloader threads...
2019-07-07 06:34:48,128 - INFO - parser - parsing result page https://www.google.com/search?q=%E7%8C%AB&ijn=0&start=0&tbs=&tbm=isch
2019-07-07 06:34:50,849 - INFO - downloader - image #1	https://img0.pclady.com.cn/pclady/pet/choice/cat/1701/6.jpg
2019-07-07 06:34:51,379 - INFO - downloader - image #2	https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/230px-Cat03.jpg
2019-07-07 06:34:51,576 - INFO - downloader - image #3	https://pic4.zhimg.com/80/v2-2e21b2dcaf9233c5697c46524b2f4aae_qhd.jpg
2019-07-07 06:34:51,611 - INFO - downloader - image #4	https://yt3.ggpht.com/a/AGF-l7_wql0rgr1h1lH6RTewb49d

In [0]:
# 犬の画像を100枚取得
crawler = GoogleImageCrawler(storage={"root_dir": "dogs"})
crawler.crawl(keyword="犬", max_num=100)

2019-07-07 06:36:59,215 - INFO - icrawler.crawler - start crawling...
2019-07-07 06:36:59,216 - INFO - icrawler.crawler - starting 1 feeder threads...
2019-07-07 06:36:59,218 - INFO - feeder - thread feeder-001 exit
2019-07-07 06:36:59,218 - INFO - icrawler.crawler - starting 1 parser threads...
2019-07-07 06:36:59,222 - INFO - icrawler.crawler - starting 1 downloader threads...
2019-07-07 06:36:59,815 - INFO - parser - parsing result page https://www.google.com/search?q=%E7%8A%AC&ijn=0&start=0&tbs=&tbm=isch
2019-07-07 06:37:00,074 - INFO - downloader - image #1	https://www.min-inuzukan.com/images/dog_img_shiba.jpg?18102302
2019-07-07 06:37:00,548 - INFO - downloader - image #2	https://grapee.jp/wp-content/uploads/33342_main1.jpg
2019-07-07 06:37:00,692 - INFO - downloader - image #3	https://dol.ismcdn.jp/mwimgs/7/1/670m/img_71c53c1d81500a1cf73a4f543e72413f27838.jpg
2019-07-07 06:37:01,530 - INFO - downloader - image #4	https://d3utmhtlcphhyc.cloudfront.net/files/topics/19_ext_01_3.png

In [0]:
from PIL import Image
import os, glob
import numpy as np
classes = ["dogs", "cats"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
Y_train = []
Y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            Y_test.append(index)
        else:
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                Y_train.append(index)

                img_trans = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trans)
                X_train.append(data)
                Y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(Y_train)
y_test  = np.array(Y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./dog_cat.npy", xy)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

classes = ["dog", "cat"]
num_classes = len(classes)
image_size = 64

"""
データを読み込む関数
"""
def load_data():
    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)
    # データを正規化
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.00005, decay=1e-6)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    model.save('./cnn.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""
def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()

Using TensorFlow backend.
W0707 06:47:11.524570 139665584744320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 06:47:11.565452 139665584744320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 06:47:11.572332 139665584744320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 06:47:11.622119 139665584744320 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0707 06:47:11.625579 1396655

Epoch 1/40
1680/1680 [==============================] - 8s 5ms/step - loss: 0.6659 - acc: 0.5952
Epoch 2/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.6099 - acc: 0.6738
Epoch 3/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.5667 - acc: 0.6940
Epoch 4/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.5272 - acc: 0.7220
Epoch 5/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.4905 - acc: 0.7625
Epoch 6/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.4768 - acc: 0.7708
Epoch 7/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.4361 - acc: 0.8018
Epoch 8/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.3974 - acc: 0.8280
Epoch 9/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.3730 - acc: 0.8411
Epoch 10/40
1680/1680 [==============================] - 2s 1ms/step - loss: 0.3580 - acc: 0.8506
Epoch 11/40
1680/1680 [======

In [0]:
from google.colab import files
uploaded = files.upload()

Saving dog1.jpg to dog1.jpg


In [0]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)

"""
dog1.jpgというファイル名の画像をGoogle Colab上にアップロードする方法は2通りあります。
1つが、下記のコードを実行し画像をアップロードする方法
from google.colab import files
uploaded = files.upload()
2つが、Colab左メニューの>アイコンを押して、目次、コード スニペット、ファイル
の3つ表示されるますが、右のファイルタブから画像をアップロードする方法です。
このファイルタブをクリックするとアップロードと更新の2つがありますが、
アップロードを押すと画像をアップロードすることが可能です。
"""

testpic     = "./dog1.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    img = img.resize(imsize)
    img = np.asarray(img)
    img = img / 255.0
    return img

def get_file(dir_path):
    filenames = os.listdir(dir_path)
    return filenames

if __name__ == "__main__":

    model = load_model(keras_param)
    img = load_image(testpic)
    prd = model.predict(np.array([img]))
    print(prd) # 精度の表示
    prelabel = np.argmax(prd, axis=1)
    if prelabel == 0:
        print(">>> 犬")
    elif prelabel == 1:
        print(">>> 猫")

[[0.8797367  0.12026333]]
>>> 犬


In [0]:
from google.colab import files
uploaded = files.upload()

Saving cat1.jpg to cat1.jpg


In [0]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)

"""
dog1.jpgというファイル名の画像をGoogle Colab上にアップロードする方法は2通りあります。
1つが、下記のコードを実行し画像をアップロードする方法
from google.colab import files
uploaded = files.upload()
2つが、Colab左メニューの>アイコンを押して、目次、コード スニペット、ファイル
の3つ表示されるますが、右のファイルタブから画像をアップロードする方法です。
このファイルタブをクリックするとアップロードと更新の2つがありますが、
アップロードを押すと画像をアップロードすることが可能です。
"""

testpic     = "./cat1.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    img = img.resize(imsize)
    img = np.asarray(img)
    img = img / 255.0
    return img

def get_file(dir_path):
    filenames = os.listdir(dir_path)
    return filenames

if __name__ == "__main__":

    model = load_model(keras_param)
    img = load_image(testpic)
    prd = model.predict(np.array([img]))
    print(prd) # 精度の表示
    prelabel = np.argmax(prd, axis=1)
    if prelabel == 0:
        print(">>> 犬")
    elif prelabel == 1:
        print(">>> 猫")

[[4.4418572e-04 9.9955577e-01]]
>>> 猫
